In [1]:
words = open("names.txt", 'r').read().splitlines()

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {c: i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {c: i for i,c in stoi.items()}
print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [99]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline



In [129]:
block_size = 3
X, Y = [], []

for w in words:
    # print(w)
    context = [0] * block_size
    for ch, in w+'.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [130]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [131]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [132]:
sum(p.nelement() for p in parameters)

3481

In [133]:
for p in parameters:
    p.requires_grad = True

In [ ]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [140]:
lri = []
lossi = []
# train
for i in range(1000):
    # forward pass
    ix = torch.randint(0, X.shape[0], (32,))
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])

    # backward pass
    lr = lrs[i]
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -lr * p.grad
    lri.append(lr)
    lossi.append(loss.item())

2.623896598815918
2.6567740440368652
2.211489677429199
2.651400566101074
2.5476040840148926
2.5895848274230957
2.806231737136841
2.5858445167541504
2.599001169204712
2.4743170738220215
2.6844820976257324
2.85396409034729
2.7649571895599365
2.5369341373443604
2.914907455444336
2.9755663871765137
2.404776096343994
2.812835931777954
2.7395613193511963
2.641242504119873
2.631462812423706
2.6557364463806152
2.8324387073516846
2.5395538806915283
2.695835828781128
2.654038667678833
2.841691493988037
2.6133205890655518
2.463557243347168
2.7550952434539795
2.5944840908050537
2.6565451622009277
2.6465744972229004
2.9554014205932617
3.157660961151123
2.4997377395629883
2.6485469341278076
2.634786605834961
3.3046202659606934
2.6305270195007324
2.666553258895874
3.232548713684082
2.735269784927368
2.6541311740875244
2.5210375785827637
2.6540095806121826
2.961725950241089
2.715193033218384
2.817639112472534
2.7460224628448486
2.6321918964385986
3.0708072185516357
2.7620344161987305
2.584081172943115

In [139]:
torch.randint(0, X.shape[0], (32,))

tensor([206255,  45550, 203592, 103875,  86324,  96803,  20402,  18070,  32394,
         42609,  78563, 170097,  22619,  62971,   5361, 198384,   6287, 156080,
         10042, 118154,  66701,  50174,  22011,  14480, 156259,  46701, 209297,
        162261,  15591,  65489,  89134, 169437])